# Prompts & Responses

1. Generate reusable prompt templates
2. Parse responses using pydantic models

In [ ]:
from typing import Literal

from langchain_anthropic.chat_models import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelName

In [ ]:
# Load API keys from .env file
api_keys = APIKeys()


# Initialize a chat model with your API key
chat_model = ChatAnthropic(
    model_name=ModelName.CLAUDE_HAIKU,
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
)

# 1. Reusable prompt template

In [ ]:
# Define a simple chat prompt template with input variables
class PoemInputs(BaseModel):
    """Poem prompt input variables.

    Attributes:
        topic (str): The topic of the poem.
        funny_or_sad (Literal["funny", "sad"]): Whether the poem should be funny or sad. Default is 'funny'.
        audience (str): The intended audience for the poem. Default is 'general'.
    """

    topic: str = Field(description="The topic of the poem")
    funny_or_sad: Literal["funny", "sad"] = Field(
        default="funny", description="Whether the poem should be funny or sad. Default is 'funny'."
    )
    audience: str = Field(default="general", description="The intended audience for the poem.")


prompt_template = ChatPromptTemplate.from_template(
    template="Please write me a short poem about {topic}. This poem should be {funny_or_sad} for {audience}.",
)
print("Prompt template created successfully.", "\n", prompt_template)

In [ ]:
# Get message prompt from the template
message_prompt = prompt_template.format_messages(
    topic="getting dressed in the morning", funny_or_sad="funny", audience="children"
)
print("Message prompt formatted successfully.", "\n", message_prompt)

In [ ]:
# Get a different message prompt from the template using PoemVariables model
prompt_variables = PoemInputs(topic="eating soup", funny_or_sad="funny", audience="6 year old boy")
message_prompt = prompt_template.format_messages(**prompt_variables.model_dump(mode="json"))
print("Message prompt formatted successfully.", "\n", message_prompt)

In [ ]:
# Prompt the model and get the response
response = chat_model.invoke(message_prompt)

print("Poem response:", "\n", response.content)

In [ ]:
# Try to create an invalid message prompt from the template
prompt_variables = PoemInputs(topic="eating soup", funny_or_sad="warm and cozy")  # This should raise a validation error
message_prompt = prompt_template.format_messages(**prompt_variables.model_dump(mode="json"))
print("Message prompt formatted successfully.", "\n", message_prompt)

# 2. Enforce structured output

In [ ]:
# Define the response model for structured output
class PoemOutputs(BaseModel):
    """Poem response output variables.

    Attributes:
        title (str): The title of the poem.
        content (str): The content of the poem.
        voice (str | None): Recommended voice for reading the poem aloud. Optional.
    """

    title: str = Field(description="The title of the poem")
    content: str = Field(description="The content of the poem.")
    voice: str | None = Field(default=None, description="Recommended voice for reading the poem aloud. Optional.")

## Option A: `PydanticOutputParser` & format instructions

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

# Create a Pydantic output parser for the response model
response_parser = PydanticOutputParser(pydantic_object=PoemOutputs)

# Create a prompt template with response format instructions
prompt_template_w_format = ChatPromptTemplate.from_template(
    template=(
        "Please write me a short poem about {topic}. This poem should be {funny_or_sad} for {audience}."
        "\n\n{format_instructions}"
    ),
    partial_variables={"format_instructions": response_parser.get_format_instructions()},
)

In [ ]:
# Create a chain combining the prompt, chat model, and response parser
chain = prompt_template_w_format | chat_model | response_parser

In [ ]:
# Invoke the chain with prompt variables
prompt_variables = PoemInputs(topic="eating soup", funny_or_sad="funny", audience="6 year old boy")
response: PoemOutputs = chain.invoke(prompt_variables.model_dump(mode="json"))

In [ ]:
# Display the structured response
print("Poem Title:", response.title)
print("\nPoem Content:", response.content)
print("\nRecommended Voice:", response.voice)

## Option B: New model `with_structured_output`

 I'd strongly recommend using `.with_structured_output()` instead of `PydanticOutputParser` for Claude models. It's more reliable because it uses Claude's native tool-calling rather than trying to parse JSON from text.

In [ ]:
# Wrap the chat model to enforce structured output
structured_model = chat_model.with_structured_output(PoemOutputs)

# Create a chain combining the (original) prompt template and the structured model
chain = prompt_template | structured_model

In [ ]:
# Invoke the chain with prompt variables
response: PoemOutputs = chain.invoke(PoemInputs(topic="playing Lego with my sister").model_dump(mode="json"))

In [ ]:
# Display the structured response
print("Poem Title:", response.title)
print("\nPoem Content:", response.content)
print("\nRecommended Voice:", response.voice)